## Imports

In [1]:
import pandas as pd
#import numpy as np

import requests
import urllib
#from bs4 import BeautifulSoup
#import itertools
#import collections
import re
#import pickle
import time
import random

#import networkx as nx
#import matplotlib.pyplot as plt

## Functions

In [2]:
# recursive function to grab the underlying html text of a webpage
def grab_url_text(theurl,remaining_tries=5):
    # These headers help to make the computer look like a legitimate web browser and not a bot
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    
    #print('Analyzing: '+theurl+' ... ',end='')
    try:
        # these lines are the meat of the program - they grab the underlying html textfile that describes the webpage
        r = urllib.request.Request(theurl,data=None,headers=headers)
        f = urllib.request.urlopen(r)
        s = f.read()
        s = s.decode('utf-8') # the string enters as a byte string which needs to be decoded into a useful string format UTF-8
        f.close()
        #print('done!')
        return s
    except: # if there is a problem, print the error and try again
        print('\n\t!!! Connection Issue')
        if remaining_tries > 0:
            time.sleep(2)
            grab_url_text(theurl,remaining_tries-1)
        else:
            raise

## Main script

### Read artists csv
#### Note: it's sorted in descending order by listeners_lastfm

In [3]:
df = pd.read_csv('artists_to_100000_listens.csv')

In [4]:
df = df.drop(labels=['artist_lastfm','country_mb','country_lastfm','tags_lastfm','scrobbles_lastfm'], axis=1)

In [7]:
df.head()

,mbid,artist_mb,tags_mb,listeners_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,rock; pop; alternative rock; british; uk; brit...,5381567,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,rock; electronic; alternative rock; british; g...,4732528,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,rock; alternative rock; 80s; 90s; rap; metal; ...,4620835,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,pop; dance; hip hop; reggae; contemporary r b;...,4558193,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,turkish; rap; american; hip-hop; hip hop; hiph...,4517997,False


In [8]:
len(df)

10911

### Scrape!

In [10]:
all_artists_mbid = df['mbid'][8049:]
num_artists = len(all_artists_mbid)
sleep_interval = random.random()

all_artist_release_credits = []
for artist_num, mbid in enumerate(all_artists_mbid):
    html = grab_url_text(f'https://musicbrainz.org/artist/{mbid}/releases')
    try:
        table = html.split('<ul class="pagination">')[1]
        table = table.split('</ul>')[0]
        result = re.findall('>\d<', table)
        result = [ele[1:-1] for ele in result if ele.startswith('>') and ele.endswith('<')]
        num_pages = int(result[-1])
    except:
        num_pages = 1
    for page_num in range(1, num_pages+1):
        try:
            time.sleep(sleep_interval)
            html = grab_url_text(f'https://musicbrainz.org/artist/{mbid}/releases?page={page_num}')
            table = html.split('<tbody role="rowgroup">')[1]
            table = table.split('</tbody>')[0]
            for line in table.split('</tr>'):
                credit = line.split('<a href="/artist', maxsplit=1)[1]
                credit = credit.split('</td>')[0]
                credit = re.split('<bdi|/bdi>', credit)
                result = [ele[1:-1] for ele in credit if ele.startswith('>') and ele.endswith('<')]
                
                entries = []
                for cell in line.split('</td>'):
                    try:
                        entry = cell.split('<td role="cell">', maxsplit=1)[1]
                    except:
                        entry = 0
                    entries.append(entry)
                num_tracks = entries[3]
                num_tracks = num_tracks.split(' + ')
                num_tracks = list(map(int, num_tracks))
                num_tracks = sum(num_tracks)
                result.append(num_tracks)
                
                if len(result) > 2:
                    with open('all_artist_release_credits.txt', 'a', encoding='utf-8') as fout:
                        for _string in result:
                            fout.write(str(_string)+'<--->')
                        #fout.writelines(result)
                        fout.write('\n')

                    all_artist_release_credits.append(result)
                #print(credit)
        except:
            pass
        print(str(artist_num+1)+f' (page: {page_num}/{num_pages}) out of {num_artists} artists done.', end='\r')
        
with open('all_artist_release_credits_full.txt', 'w', encoding='utf-8') as fout:
    for _list in all_artist_release_credits:
        for _string in _list:
            fout.write(str(_string))
            fout.write('<--->')
        fout.write('\n')

2600 (page: 1/1) out of 2862 artists done.
	!!! Connection Issue


In [54]:
for i, line in enumerate(all_artist_release_credits):
    print(i, line)

0 ['Coldplay', 'コールドプレイ', 15]
1 ['Coldplay', 'Kylie Minogue', 1]
2 ['Coldplay', 'JAY-Z', 1]
3 ['Drake', 'Coldplay', 10]
4 ['Coldplay', 'Swedish House Mafia', 2]
5 ['Coldplay', 'Rihanna', 1]
6 ['Coldplay', 'Rihanna', 4]
7 ['Coldplay', 'Rihanna', 4]
8 ['Coldplay', 'Rihanna', 4]
9 ['Coldplay', 'Rihanna', 4]
10 ['Cat Power', 'Coldplay', 1]
11 ['Coldplay', 'Alan Walker', 1]
12 ['Coldplay', 'Beyoncé', 5]
13 ['The Chainsmokers', 'Coldplay', 1]
14 ['The Chainsmokers', 'Coldplay', 6]
15 ['The Chainsmokers', 'Coldplay', 6]
16 ['Coldplay', 'The Chainsmokers', 1]
17 ['DJ Shadow', 'Radiohead', 2]
18 ['Aquasky', 'Radiohead', 1]
